In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import os,random, math, psutil, pickle

import pandas_profiling as pdp

from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

import category_encoders as ce


In [2]:
train = pd.read_csv('./input/bank_marketing_train.csv')

In [3]:
train.shape

(33744, 20)

In [4]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [5]:
train.head()

,age,job,marital,education,default,housing,loan,contact,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,56,services,married,high.school,no,no,yes,telephone,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,45,services,married,basic.9y,unknown,no,no,telephone,mon,198,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,59,admin.,married,professional.course,no,no,no,telephone,mon,139,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [6]:
train.job.value_counts()

admin.           8536
blue-collar      7591
technician       5529
services         3255
management       2359
retired          1432
entrepreneur     1203
self-employed    1130
housemaid         882
unemployed        822
student           735
unknown           270
Name: job, dtype: int64

In [7]:
train.education.value_counts()

university.degree      9971
high.school            7830
basic.9y               4992
professional.course    4265
basic.4y               3428
basic.6y               1819
unknown                1424
illiterate               15
Name: education, dtype: int64

In [8]:
education_rank = {
    "unknown": 0,
    "illiterate": 1,
    "basic.4y" : 2,
    "basic.6y"  :3,
    "basic.9y":4,
    "high.school":5, 
    "professional.course":6 ,
    "university.degree":7
}

In [9]:
train["nr.employed"].value_counts()

5228.1    13276
5099.1     6987
5191.0     6389
5195.8     3014
5076.2     1362
5017.5      880
4991.6      633
5008.7      528
4963.6      524
5023.5      143
5176.3        8
Name: nr.employed, dtype: int64

In [10]:
#pdp.ProfileReport(train)

In [11]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx',
       'euribor3m', 'nr.employed', 'y'],
      dtype='object')

# 前処理

In [12]:
# durationをdropする
#numeric = [ 'cons.price.idx', 'cons.conf.idx', 'age','campaign','pdays', 'previous'  ]
numeric = ['age','campaign','pdays', 'previous'  ]
categoricals = ['job', 'marital',  'default', 'housing', 'loan',
#       'contact', 'day_of_week',  'poutcome', 'emp.var.rate', 'euribor3m', 'nr.employed']
     # 'contact', 'day_of_week',  'poutcome', 'euribor3m']
      'education','contact', 'day_of_week',  'poutcome']#, 'euribor3m']


#rank_categoricals = []
rank_categoricals = ["education"]

feat_cols = numeric + categoricals


In [13]:
def preprocessing ( df ) :
    df.drop(['duration'], axis=1, inplace=True)
    df["y"].replace("no",0, inplace=True)
    df["y"].replace("yes",1, inplace=True)
    ce_oe = ce.OrdinalEncoder(cols=categoricals,handle_unknown='impute')
    df = ce_oe.fit_transform(df)
    return df


In [14]:
import dumper

def exp_val_func (y_pred , data):
    valid_y = data.get_label().tolist()
    tmp_exp_val = 0
    pred_yes = 0
    
    y_pred = y_pred.round(0).astype("u1") 
    
    dumper.dump(y_pred)
    print(len(valid_y))
    for i in range (len(valid_y)) :
        if y_pred[i]:
            tmp_exp_val -= 1
            pred_yes += 1
            if valid_y[i]:
                tmp_exp_val += 10
                
       # exp_val += tmp_exp_val
    print(pred_yes)
#         if i % 1000 == 999 :
#             print(tmp_exp_val)
    
    return 'exp_val', tmp_exp_val, True

In [15]:
train = preprocessing(train)

# モデル作成

In [16]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y'],
      dtype='object')

# ポイント

In [17]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            #'metric': {'binary_logloss'},
            'metric': {'None'},
            'subsample': 0.8, # 汎化性能
            'subsample_freq': 1,
            'learning_rate': 0.01, # 0.1で試して、0.01で最終
            'num_leaves': 8, # 2^3 ~ 2^6
            'feature_fraction': 0.9,
            'lambda_l1': 2,  
            'lambda_l2': 2,
            'min_data_in_leaf': 20, # 汎化性能
            
            #'label_gain':[1,10]
            'scale_pos_weight':9 # 重みづけ
            #'device':'gpu'
            }

# folds = 5
# seed = 666
# kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=seed)

folds = 5
seed = 666

kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
#kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

In [18]:
#(train_index, val_index) = kf.split(train, train["y"])
# hoge = kf.split(train, train["y"])

i =0
for train_index, val_index in kf.split(train, train['y']):
    train_X = train[feat_cols].iloc[train_index] 
    train_y = train["y"].iloc[train_index] 
    val_X = train[feat_cols].iloc[val_index]
    val_y = train["y"].iloc[val_index]
    i = i+1
    print(1)
    break



1


In [19]:
#train_X

#train_X_new = pd.DataFrame(train_X, index=train_X_origin.index , columns=train_X_origin.columns)

# ポイント

In [20]:
# from imblearn.over_sampling import SMOTE

# train_X_origin = train_X.copy()
# train_y_origin = train_y.copy()

# sm = SMOTE(random_state=42)
# train_X, train_y = sm.fit_sample(train_X, train_y)

# p = list(zip(train_X, train_y))
# random.shuffle(p)
# train_X, train_y = zip(*p)


In [21]:
# train_X = pd.DataFrame(train_X, columns=train_X_origin.columns)
# train_y = np.array(train_y)

In [22]:
lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)




In [23]:
train_y.sum()
train_y.shape

pd.DataFrame(train_y).tail(10)

,y
33733,0
33734,0
33735,0
33736,1
33737,0
33738,0
33739,1
33740,0
33742,1
33743,0


In [24]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100,
                
                feval = exp_val_func,
               )

<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
Training until validation scores don't improve for 100 rounds.
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([

<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'26995
0
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)'6749
0
<str at 0x1a2226687

401
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
1772
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 1], dtype=uint8)'6749
404
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
1778
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 1], dtype=uint8)'6749
405
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
1797
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 1], dtype=uint8)'6749
416
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
1821
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 1], dtype=uint8)'6749
429
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
1833
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 1], dtype=uint8)'6749
431
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
1857
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 0, 0, 1], dtype=uint8)'6749
437
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype

952
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
4090
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
972
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
4157
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
990
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
4247
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1010
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
4353
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1037
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
4414
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1060
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
4539
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1084
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], d

7916
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1884
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
7967
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1895
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
8009
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1909
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
8076
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1918
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
8144
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1934
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
8195
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'6749
1944
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
8234
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1]

2311
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
9606
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2314
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
9631
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2321
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
9638
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2321
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
9649
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2324
<str at 0x10c886a50>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
9670
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2327
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
9683
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2332
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1],

2459
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
10185
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2458
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
10198
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2463
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
10199
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2461
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
10200
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2468
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
10210
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2469
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 0, 1], dtype=uint8)'26995
10211
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1, 1, 1], dtype=uint8)'6749
2473
<str at 0x1a22266870>: 'array([0, 0, 0, ..., 1,

In [25]:
fi = pd.DataFrame(  {0: train_X.columns ,  1: gbm.feature_importance()})
fi

,0,1
0,age,483
1,campaign,235
2,pdays,226
3,previous,36
4,job,297
5,marital,45
6,default,153
7,housing,11
8,loan,26
9,education,167


In [26]:
from sklearn.metrics import confusion_matrix
# y_true = [0, 0, 0, 0, 1, 1, 1, 0, 1, 0]   # 実際の値 (0:スパムでない, 1:スパム)
# y_pred = [0, 0, 0, 0, 1, 1, 1, 1, 0, 1]   # 識別結果 (0:スパムでない, 1:スパム)
y_pred = gbm.predict(val_X)


In [27]:
#y_pred.tolist()

In [28]:
val_y.tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [29]:
val_y_c = val_y.tolist()
y_pred_c =  y_pred.tolist()


# ポイント

In [30]:
# labels = ["not cvr", "cvr"]
#confusion_matrix( val_y_c, y_pred.round(), labels=labels, index=labels)

from pycm import ConfusionMatrix
cm = ConfusionMatrix(actual_vector=val_y_c, predict_vector=y_pred.round(0).astype("u1") )  
cm.print_matrix()

Predict    0          1          
Actual
0          4177       1855       

1          251        466        




In [31]:
test = pd.read_csv('./input/bank_marketing_test.csv')

In [32]:
test = preprocessing(test)

In [33]:
test_X = test[feat_cols]
#test_X

In [34]:

test_y = np.array(test["y"])
test_y

array([0, 0, 0, ..., 0, 0, 0])

In [35]:
y_test_pred = gbm.predict(test_X)

In [36]:
test_y_c = test_y.tolist()
y_test_pred_c =  y_test_pred.tolist()

In [37]:
from pycm import ConfusionMatrix
cm_t = ConfusionMatrix(actual_vector=test_y_c, predict_vector=y_test_pred.round(0).astype("u1") )  
cm_t.print_matrix()

Predict    0          1          
Actual
0          4992       2308       

1          316        621        




# 何がダメだったか？

→もっと電話かけた方が良い！！
→多分、なんらか環境要因的にテストはトレインより低かった